In [133]:
#ignore, used to look up OG contents of input file
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore

path = './data/'
filename_read = os.path.join(path,'heart.csv')
df = pd.read_csv(filename_read)


In [134]:
#Useful functions
import os
import pandas as pd

path = "./data/"

filename_read = os.path.join(path, "heart.csv")
df = pd.read_csv(filename_read)

#Randomize the data
np.random.seed(50)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)


headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'data Val' : field,
        'mean' : df[field].mean(),
        'var' : df[field].var(),
        'sdev' : df[field].std()
    })
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

    


In [129]:
#Classification Neural Network
import os
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import requests 
import io 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from keras.models import Sequential 
from keras.layers.core import Dense, Activation 
from keras.callbacks import EarlyStopping

path = "./data/"

filename_read = os.path.join(path, "heart.csv") 
df = pd.read_csv(filename_read)

#one-hot coding for catagory classification data 
one_hot = pd.get_dummies(df['sex']) 
df = df.drop('sex', axis = 1) 
df = df.join(one_hot) 
df.rename(columns = { 0: "female",
                     1: "male"}, 
          inplace = True) 
one_hot = pd.get_dummies(df['fbs']) 
df = df.drop('fbs', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "fbs-no",
                     1: "fbs-yes"}, 
          inplace = True) 

one_hot = pd.get_dummies(df['exang'])
df = df.drop('exang', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "exang-no",
                     1: "exang-yes"},
          inplace = True)

#Normalize the real number data
df['age'] = zscore(df['age']) 
df['cp'] = zscore(df['cp']) 
df['trestbps'] = zscore(df['trestbps']) 
df['chol'] = zscore(df['chol']) 
df['restecg'] = zscore(df['restecg']) 
df['age'] = zscore(df['age']) 
df['thalach'] = zscore(df['thalach']) 
df['oldpeak'] = zscore(df['oldpeak']) 
df['slope'] = zscore(df['slope']) 
df['ca'] = zscore(df['ca']) 
df['thal'] = zscore(df['thal'])

df = df.reindex(np.random.permutation(df.index))#shuffle the data 
df.reset_index(inplace=True, drop=True)



#Pre-segment the data
mask = np.random.rand(len(df)) < 0.7
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])

# Use the to_xy() to convert the pre-segmented data into x,y corridnates used by tensorflow
x,y = to_xy(trainDF, "target")
x1,y1 = to_xy(validationDF, "target")

#split into train/test 
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.25, random_state=42)#Train on 70% Tes on 30

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))#IHidden layer1
model.add(Dense(5,activation='relu')) #Hidden Layer
model.add(Dense(y.shape[1],activation='softmax')) #Output Layer
model.compile(loss='categorical_crossentropy', optimizer='adam') 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,patience=5, verbose=1, mode='auto')#Stop, when improvments arn't made for 5 turns              

model.fit(x,y, validation_data=(x_test,y_test),callbacks=[monitor], verbose=2,epochs=500)

Train on 216 samples, validate on 54 samples
Epoch 1/500
 - 2s - loss: 0.7490 - val_loss: 0.7321
Epoch 2/500
 - 0s - loss: 0.7278 - val_loss: 0.7139
Epoch 3/500
 - 0s - loss: 0.7093 - val_loss: 0.6954
Epoch 4/500
 - 0s - loss: 0.6902 - val_loss: 0.6770
Epoch 5/500
 - 0s - loss: 0.6731 - val_loss: 0.6594
Epoch 6/500
 - 0s - loss: 0.6559 - val_loss: 0.6442
Epoch 7/500
 - 0s - loss: 0.6405 - val_loss: 0.6291
Epoch 8/500
 - 0s - loss: 0.6247 - val_loss: 0.6155
Epoch 9/500
 - 0s - loss: 0.6115 - val_loss: 0.6021
Epoch 10/500
 - 0s - loss: 0.5975 - val_loss: 0.5893
Epoch 11/500
 - 0s - loss: 0.5849 - val_loss: 0.5770
Epoch 12/500
 - 0s - loss: 0.5725 - val_loss: 0.5652
Epoch 13/500
 - 0s - loss: 0.5595 - val_loss: 0.5547
Epoch 14/500
 - 0s - loss: 0.5479 - val_loss: 0.5445
Epoch 15/500
 - 0s - loss: 0.5374 - val_loss: 0.5347
Epoch 16/500
 - 0s - loss: 0.5262 - val_loss: 0.5251
Epoch 17/500
 - 0s - loss: 0.5162 - val_loss: 0.5160
Epoch 18/500
 - 0s - loss: 0.5056 - val_loss: 0.5068
Epoch 19/5

Epoch 154/500
 - 0s - loss: 0.2452 - val_loss: 0.2648
Epoch 155/500
 - 0s - loss: 0.2444 - val_loss: 0.2636
Epoch 156/500
 - 0s - loss: 0.2433 - val_loss: 0.2626
Epoch 157/500
 - 0s - loss: 0.2429 - val_loss: 0.2621
Epoch 158/500
 - 0s - loss: 0.2422 - val_loss: 0.2626
Epoch 159/500
 - 0s - loss: 0.2414 - val_loss: 0.2601
Epoch 160/500
 - 0s - loss: 0.2403 - val_loss: 0.2590
Epoch 161/500
 - 0s - loss: 0.2397 - val_loss: 0.2584
Epoch 162/500
 - 0s - loss: 0.2387 - val_loss: 0.2579
Epoch 163/500
 - 0s - loss: 0.2384 - val_loss: 0.2570
Epoch 164/500
 - 0s - loss: 0.2370 - val_loss: 0.2564
Epoch 165/500
 - 0s - loss: 0.2361 - val_loss: 0.2558
Epoch 166/500
 - 0s - loss: 0.2352 - val_loss: 0.2552
Epoch 167/500
 - 0s - loss: 0.2350 - val_loss: 0.2544
Epoch 168/500
 - 0s - loss: 0.2339 - val_loss: 0.2530
Epoch 169/500
 - 0s - loss: 0.2339 - val_loss: 0.2522
Epoch 170/500
 - 0s - loss: 0.2323 - val_loss: 0.2514
Epoch 171/500
 - 0s - loss: 0.2319 - val_loss: 0.2500
Epoch 172/500
 - 0s - loss: 

In [130]:
from sklearn import metrics
import tensorflow as tf

#Run the model on the test sample
pred1 = model.predict(x_test)
pred1 = np.argmax(pred1, axis=1)
print("Shape: {}".format(pred1.shape))
print(pred1)

y_compare = np.argmax(y_test, axis=1)
score = metrics.accuracy_score(y_compare, pred1)
print("Accuracy: {}".format(score))

Shape: (54,)
[0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 1]
Accuracy: 0.9074074074074074


In [131]:
#Run model on validation sample
validation_pred = model.predict(x1)
predicted_hd = np.argmax(validation_pred, axis=1)

expected_hd = np.argmax(y1, axis=1)

score = metrics.accuracy_score(expected_hd, predicted_hd)
comparison = (predicted_hd == expected_hd)
print("Predictions: {}".format(predicted_hd))
print("Expected: {}".format(expected_hd))
print("Comparison: {}".format(comparison))
print("Accuracy: {}".format(score))

Predictions: [1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1
 1 1 0 0 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0]
Expected: [1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1
 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1
 1 1 1 0 1 1 0 1 1 1 0 1 0]
Comparison: [ True  True  True False  True  True False  True  True  True False False
  True False  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True False  True  True  True  True False
  True  True  True False  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True False  True False  True  True  True
  True  True  True  True  True False  True  True False  True  True False
  True  True  True]
Accuracy: 0.8160919540229885


In [132]:
#Calculate log loss (Inaccuracy of predictions) 
from IPython.display import display

# Dont display numpy in SI notation
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
display(pred[0]*100)

print("As percent probability")
display(pred[0:5]*100)

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions


array([97.2077,  2.7923], dtype=float32)

As percent probability


array([[97.2077,  2.7923],
       [99.2985,  0.7015],
       [ 0.6714, 99.3286],
       [25.564 , 74.436 ],
       [ 6.8697, 93.1303]], dtype=float32)

Log loss score: 0.1794267597132333
